SQL链或代理中最容易出错的部分之一就是编写有效且安全的SQL查询。在本指南中，我们将讨论一些用于验证查询和处理无效查询的策略。

我们将涵盖以下内容：

- 在查询生成过程中添加一个“查询验证”步骤；
- 提示工程以减少错误的发生率。

下面的示例将使用与Chinook数据库连接的SQLite。

In [32]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../../example-data/Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM Artist LIMIT 10;"))

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]


# 查询检查器
也许最简单的策略是让模型本身检查原始查询中常见的错误。假设我们有以下 SQL 查询链：

In [33]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="qwen-plus", temperature=0.0)

In [34]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

如果我们希望验证其输出，可以通过扩展链并添加第二个提示和模型调用来实现：

In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """
Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins
If there are any of the above mistakes, rewrite the query.
If there are no mistakes, just reproduce the original query with no further commentary.
Output the final SQL query only.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)

validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [42]:
query = full_chain.invoke(
    {
        "question": "What's the average Invoice from an American customer whose Fax is missing"
    }
)
print(query)

SELECT AVG("Total") AS "AverageInvoice" FROM "Invoice" WHERE "BillingCountry" = 'USA' AND "CustomerId" IN (SELECT "CustomerId" FROM "Customer" WHERE "Fax" IS NULL)


执行查询：

In [43]:
db.run(query)

'[(5.866349206349206,)]'

# 方法的局限性

这种方法的明显缺点是，我们需要进行两次模型调用来生成查询。为了解决这个问题，我们可以尝试在单次模型调用中完成查询生成和查询检查。

In [55]:
system = """You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Use format:

First draft: FIRST_DRAFT_QUERY
Final answer: FINAL_ANSWER_QUERY
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")]
).partial(dialect=db.dialect)


def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]


chain = create_sql_query_chain(llm, db, prompt=prompt) | parse_final_answer
prompt.pretty_print()

================================ System Message ================================

You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then 

In [56]:
query = chain.invoke(
    {
        "question": "What's the average Invoice from an American customer whose Fax is missing"
    }
)
print(query)

SELECT AVG("Total") AS AverageInvoice FROM "Invoice" WHERE "CustomerId" IN (SELECT "CustomerId" FROM "Customer" WHERE "Country" = 'USA' AND "Fax" IS NULL);


In [57]:
db.run(query)

'[(5.866349206349206,)]'